# Tree-based Classifiers


### Set Up

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

# model_storage
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Model Training and Evaluation
from sklearn.model_selection import StratifiedKFold, GridSearchCV, cross_val_predict #train_test_split,  
from sklearn.preprocessing import MinMaxScaler #, StandardScaler
from imblearn.pipeline import Pipeline as ImbPipeline #from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import make_scorer, balanced_accuracy_score
import matplotlib.pyplot as plt

# Set up the root directory for imports
import pyrootutils
root = pyrootutils.setup_root(
    search_from=os.path.abspath(''),
    indicator=[".git"],
    pythonpath=True,
    dotenv=True,
)

from utils.file_management.config_loader import load_yaml, process_config_values
from utils.file_management.file_manager import FileManager

from utils.model_utils.data_utils import load_data, visualize_data, split_data, MultiClassToBinaryConverter
from utils.model_utils.eval_utils import eval_report, load_json
from utils.model_utils.shap_utils import shap_analysis_pipeline # Feature Importance


In [ ]:
# Load yaml file with dataset information
config_path = str(root) + '/src/config/LBP_cohort.yaml'
config = process_config_values(load_yaml(config_path))
# For debug
# print(config.keys())

# Load paths to data
PlumsFiles = FileManager(config.get('file_directory'))

# --- DEFINE VARIABLES ---
# --- dataset variables ---
# Path to preprocessed data
master_data_path         = PlumsFiles.get_datapath('model_output_dir').replace('MODEL','classification_1class_meds').replace('INDEPENDENT_VAR','master_data_for_analysis.csv') 
master_encoded_data_path = PlumsFiles.get_datapath('model_output_dir').replace('MODEL','classification_1class_meds').replace('INDEPENDENT_VAR','master_numerical_data_for_analysis.csv') 
save_path_reference = PlumsFiles.get_datapath('model_output_dir')

# Data options
time_frame_list = ['2012_to_2024', '2012_to_2018', '2019_to_2024', '2012_to_2016', '2017_to_2019', '2020_to_2024'] # Options (1 set, 2 sets, 3 sets): ['2012_to_2024', '2012_to_2018', '2019_to_2024', '2012_to_2016', '2017_to_2019', '2020_to_2024']
time_frame_list = ['2012_to_2024']
data_type_list = ['tabular_text','tabular','text', 'diagnoses_tabular', 'diagnoses_text', 'psychosocial_tabular'] # TODO: Define subset of data

# --- OVO and OVR variables ---
# Define columns for creating the dataframe and data loader
y_col = 'interventiontype'

# Define label combinations for analysis
labels_dict = {
    0:'none',
    1:'nsaids',
    2: 'opioids',
    }
# Note: The label encoding maps label1 to 0 and label2 to 1
paired_labels = [(0,1), (0,2), (1,2), ('rest', 0),('rest', 1),('rest', 2)] #full paired labels
#paired_labels = list(combinations(df_data[outcome_column].unique(), 2))

# --- training variables ---
n_splits = 5

model_storage = {
    'decision_tree_best': {
        'model': DecisionTreeClassifier(random_state=0),
        'hyperparams': {
            'model__criterion': ['gini', 'entropy'],
            'model__splitter': ['best', 'random'],
            'model__max_depth': [None, 2, 4, 6, 8],
            'model__min_samples_split': [2, 5, 10],
            'model__min_samples_leaf': [1, 2, 4],
            'model__class_weight': ['balanced'],  
        },
    },
    'random_forest_best': {
        'model': RandomForestClassifier(random_state=0),
        'hyperparams': {
            'model__n_estimators': [50, 100, 200],
            'model__max_depth': [10, 15, 20],
            'model__min_samples_split': [2, 5, 10],
            'model__min_samples_leaf': [1, 2, 4],
            'model__bootstrap': [True, False],
            'model__class_weight': ['balanced', 'balanced_subsample', None],  
        },
    },
    'bagging_best': {
        'model': BaggingClassifier(estimator=RandomForestClassifier(random_state=0), random_state=0),
        'hyperparams': {
            'model__n_estimators': [10, 50, 100],
            'model__max_samples': [0.5, 0.7, 1.0],
            'model__max_features': [0.5, 0.7, 1.0],
            'model__bootstrap': [True, False],
            'model__bootstrap_features': [True, False],
            
        },
    },
    'adaboost_best': {
        'model': AdaBoostClassifier(estimator=DecisionTreeClassifier(random_state=0), random_state=0),
        'hyperparams': {
            'model__n_estimators': [50, 100, 180, 200],
            'model__learning_rate': [0.01, 0.1, 0.5, 1.0],
        }
    },
    'XGBoost_best': {
        'model': XGBClassifier(random_state=0),
        'hyperparams':{
            'model__objective': ['binary:logistic'],
            'model__max_depth': [3, 4, 5, 6],
            'model__learning_rate': [0.01, 0.1, 0.5, 1.0],
            'model__n_estimators': [50, 100, 150, 200],
            'model__lambda': [0, 10, 50, 100],
            'model__alpha': [0, 10, 50, 100],
            'model__scale_pos_weight': [1, 1.4, 1.7, 3.4, 4.8, 8.2],  # Control the balance of positive and negative weights, 
        }
    },
    # 'LightGBM_best': {
    #     'model': LGBMClassifier(random_state=0),
    #     'hyperparams':{
    #         'model__num_leaves': [31, 50],
    #         'model__max_depth': [3, 4, 5, 6],
    #         'model__learning_rate': [0.05, 0.1, 0.2],
    #         'model__n_estimators': [50, 100],
    #         'model__scale_pos_weight': [1, 1.4, 1.7, 3.4, 4.8, 8.2],
    #         'model__reg_lambda': [0, 10, 50, 100],
    #         'model__reg_alpha': [0, 10, 50, 100],
    #     }
    # },
}




# Training

### Train Models
Finetune tree-based models.<br>
Choose the best finetuned model from architecture options based on val set metric balanced_acc_adjusted_for_chance.<br>
Calculate performance metrics on the test set.<br>

In [ ]:
# --- Main Training Code ---
for time_frame in time_frame_list[0]:
    for data_type in data_type_list[0:4]:
        analysis_path = save_path_reference.replace('MODEL','classification_1class_meds').replace('INDEPENDENT_VAR',f'v5/{data_type}/{time_frame}') 
        os.makedirs(analysis_path, exist_ok=True)

        # Load, select, and visualize data
        df_categorical, df_data, continuous =  load_data(master_data_path, master_encoded_data_path, analysis_path, data_type, time_frame)
        visualize_data(df_categorical, analysis_path)

        # Prepare data for training
        df_dev, df_test = split_data(df_data, y_col)

        categorical = [col for col in df_data.columns if col not in continuous]
        BinaryLabelLoader = MultiClassToBinaryConverter(y_col, categorical_cols=categorical)

        # Define k-fold cross-validation
        kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)


        analysis_path2 = analysis_path 
        os.makedirs(analysis_path2, exist_ok=True)
        # Create a scorer object using balanced_accuracy_score
        balanced_accuracy_scorer = make_scorer(balanced_accuracy_score, adjusted=True)

        # --- MODEL TESTING AND HYPERPARAMETER TUNING ---
        # Iterate over each label pair and predictor variable
        for X_train, y_train, names in BinaryLabelLoader.process_data_stream(df_dev, paired_labels):
            X_train = np.array(X_train)
            y_train = np.array(y_train)
            label_0, label_1, input_variable = names.values()
            label_0_name = 'rest' if label_0 == 'rest' else labels_dict[label_0]
            label_1_name = labels_dict[label_1] 

            # Select the best model
            results = []
            models = {}
            
            for model_name, model_info in model_storage.items():
                # Directory to save project analysis
                model_save_path = f'{analysis_path2}/pred_{label_0_name}_vs_{label_1_name}/{model_name}'

                save_path = model_save_path + f'/metrics.json'
                if os.path.exists(save_path):
                    metrics_dict = load_json(save_path)
                    results.append(metrics_dict)
                    continue
                    
                # Pipeline with feature scaling and model
                pipeline = ImbPipeline([
                    ('scaler', MinMaxScaler()),  # Standardization
                    ('oversample', SMOTE(random_state=42)),  # Oversample the minority class
                    ('model', model_info['model']) # Model
                ])
                
                # Perform Grid Search with Cross-Validation
                grid_search = GridSearchCV(
                    estimator=pipeline,
                    param_grid=model_info['hyperparams'],
                    cv=kf,
                    scoring=balanced_accuracy_scorer,  # Options 'f1', 'roc_auc', 'precision', 'recall', 'balanced_accuracy' 
                                                       # f1 we care more about + detection, balanced_acc care about + and - class detection
                    n_jobs=-1,
                    verbose=0
                )

                start_time = datetime.now()
                print(f"Starting Grid Search for {data_type} {time_frame} {model_name} {label_0_name}_vs_{label_1_name}. {start_time.strftime('%H:%M')}")
                grid_search.fit(X_train, y_train)
                print("Total runtime:", (datetime.now() - start_time), "HH:MM:SS")

                # Best parameters and model
                best_pipeline = grid_search.best_estimator_
                best_params = grid_search.best_params_
                print(f"Best parameters for {model_name} {label_0_name}_vs_{label_1_name}: {best_params}")
                
                # Use cross_val_predict for evaluation
                y_pred = cross_val_predict(best_pipeline, X_train, y_train, cv=kf, method='predict')
                y_pred_prob = cross_val_predict(best_pipeline, X_train, y_train, cv=kf, method='predict_proba')[:, 1]
                
                run_info = {
                    'data_type': data_type,
                    'time_frame': time_frame,
                    'label_0': label_0_name,
                    'label_1': label_1_name,
                    'inputs': input_variable,
                    'model_name': model_name,
                    'best_params': best_params,
                    'set': 'validation',
                }

                metrics = eval_report(y_train, y_pred, y_pred_prob, model_save_path, run_info)
                results.append(metrics)
                models[model_name] = best_pipeline

            # Test set performance
            test_save_path = f'{analysis_path2}/pred_{label_0_name}_vs_{label_1_name}/best_model_test_set'
            if os.path.exists(test_save_path):
                continue
            if models == {}:
                continue
            
            # Select rows with the highest AUC for each Output value
            results_df = pd.DataFrame(results)
            best_models_df = results_df.loc[results_df.groupby('labels')['balanced_accuracy_adjusted_chance'].idxmax()].reset_index()
            if len(best_models_df)>1:
                raise('too many matches')
                
            model_name = best_models_df['model'][0]
            if model_name not in models.keys():
                continue
            best_pipeline = models[model_name]
            
            for X_test, y_test, names in BinaryLabelLoader.process_data_stream(df_test, [(label_0, label_1)]):
                X_test = np.array(X_test)
                y_test = np.array(y_test)
        
                # Use cross_val_predict for evaluation
                y_pred = best_pipeline.predict(X_test)
                y_pred_prob = best_pipeline.predict_proba(X_test)[:, 1]
                
                run_info = {
                    'data_type': data_type,
                    'time_frame': time_frame,
                    'label_0': label_0_name,
                    'label_1': label_1_name,
                    'inputs': input_variable,
                    'model_name': model_name,
                    'best_params': best_params,
                    'set': 'test',
                }
                
                metrics = eval_report(y_test, y_pred, y_pred_prob, test_save_path, run_info)


### Validation Set Metrics Summary
Create a csv file that compiles the validation performance metrics from each finetuned model architecture. 

In [ ]:
# --- MODEL SUMMARY (VALIDATION SET) ---
results = []

for time_frame in time_frame_list:
    for data_type in data_type_list:
        analysis_path = save_path_reference.replace('MODEL','classification_1class_meds').replace('INDEPENDENT_VAR',f'v5/{data_type}/{time_frame}') 

        # Load, select, and visualize data
        _, df_data, continuous =  load_data(master_data_path, master_encoded_data_path, analysis_path, data_type, time_frame)
        
        # Prepare data for training
        df_dev, df_test = split_data(df_data, y_col)

        categorical = [col for col in df_data.columns if col not in continuous]
        BinaryLabelLoader = MultiClassToBinaryConverter(y_col, categorical_cols=categorical)
        
        # Iterate over each label pair and predictor variable
        for X_train, y_train, names in BinaryLabelLoader.process_data_stream(df_dev, paired_labels):
            X_train = np.array(X_train)
            y_train = np.array(y_train)
            label1, label2, input_variable = names.values()
            label_name1 = 'rest' if label1 == 'rest' else labels_dict[label1]
            label_name2 = labels_dict[label2] 

            for model_name, model_info in model_storage.items():
                # Directory to save project analysis
                model_save_path = f'{analysis_path}/pred_{label_name1}_vs_{label_name2}/{model_name}'
                #print(model_save_path)

                save_path = model_save_path + '/metrics.json'
                metrics_dict = load_json(save_path)
                results.append(metrics_dict)
    
# Convert results to a DataFrame for better visualization
print("Hyperparameter Tuning Results:")
results_df = pd.DataFrame(results)
print(len(results_df))
#print(results_df)

# Save tree results summary
comparison_dir = save_path_reference.replace('MODEL','classification_1class_meds').replace('INDEPENDENT_VAR',f'v5')
csv_path = f'{comparison_dir}/tree_models_validation_summary.csv'
print('run summary: ', csv_path)
results_df.to_csv(csv_path)


# Eval Test Set

### Test Set Performance Metrics
For each best finetuned model, calculate the test set performance metrics

In [ ]:
# --- TEST SET ---
def map_label_name_to_num(labels_dict, label_name):
    return [num for num, name in labels_dict.items() if name == label_name][0]

for time_frame in time_frame_list:
    for data_type in data_type_list:
        analysis_path = save_path_reference.replace('MODEL','classification_1class_meds').replace('INDEPENDENT_VAR',f'v3/{data_type}/{time_frame}') 
        os.makedirs(analysis_path, exist_ok=True)

        # Load, select, and visualize data
        _, df_data, continuous =  load_data(master_data_path, master_encoded_data_path, analysis_path, data_type, time_frame)

        # Prepare data for training
        df_dev, df_test = split_data(df_data, y_col)

        categorical = [col for col in df_data.columns if col not in continuous]
        BinaryLabelLoader = MultiClassToBinaryConverter(y_col, categorical_cols=categorical)

        analysis_path2 = analysis_path 
        os.makedirs(analysis_path2, exist_ok=True)
        # Create a scorer object using balanced_accuracy_score
        balanced_accuracy_scorer = make_scorer(balanced_accuracy_score, adjusted=True)

        # Iterate over each label pair and predictor variable
        for X_train, y_train, names in BinaryLabelLoader.process_data_stream(df_dev, paired_labels):
            X_train = np.array(X_train)
            y_train = np.array(y_train)
            label_0, label_1, input_variable = names.values()
            label_0_name = 'rest' if label_0 == 'rest' else labels_dict[label_0]
            label_1_name = labels_dict[label_1] 

            # Select the best model
            results = []

            for model_name, model_info in model_storage.items():
                # Directory to save project analysis
                model_save_path = f'{analysis_path2}/pred_{label_0_name}_vs_{label_1_name}/{model_name}'
                #print(model_save_path)

                save_path = model_save_path + '/metrics.json'
                metrics_dict = load_json(save_path)
                results.append(metrics_dict)
        
            # Convert results to a DataFrame for better visualization
            print("Hyperparameter Tuning Results:")
            results_df = pd.DataFrame(results)
            #print(results_df)

            # Save tree results summary
            csv_path = f'{analysis_path2}/pred_{label_0_name}_vs_{label_1_name}/tree_models_summary.csv'
            print('run summary: ', csv_path)
            results_df.to_csv(csv_path)

            # Select rows with the highest AUC for each Output value
            best_models_df = results_df.loc[results_df.groupby('labels')['balanced_accuracy_adjusted_chance'].idxmax()].sort_values('labels').reset_index()
            #print(best_models_df)

            for index, row in best_models_df.iterrows():
                model_name = row['model']
                # DEBUG
                if "XGBoost" in model_name:
                    continue
                hyperparams = row['best_parameters']
                hyperparams = {key:[value] for key, value in hyperparams.items()}
                label_0_name = row['label_0']
                label_1_name = row['label_1']
                model_info = model_storage[model_name]

                label_0 = 'rest' if label_0_name == 'rest' else map_label_name_to_num(labels_dict, label_0_name)
                label_1 = map_label_name_to_num(labels_dict, label_1_name)
                
                # Directory to save project analysis
                model_save_path = f'{analysis_path2}/pred_{label_0_name}_vs_{label_1_name}/best_model_test_set'

                if os.path.exists(f'{model_save_path}/metrics.json'):
                    metrics_dict = load_json(f'{model_save_path}/metrics.json')
                    #if metrics_dict['model'] == model_name:
                    #    continue
                if os.path.exists(f'{model_save_path}/SHAP_test/SHAP_heatmap.png'):
                    continue

                for X_train, y_train, names in BinaryLabelLoader.process_data_stream(df_dev, [(label_0, label_1)]):
                    # SHAP Variables
                    X_SHAP_train = X_train.copy(deep=True)

                    # Training Variables
                    X_train = np.array(X_train)
                    y_train = np.array(y_train)

                    # Pipeline with feature scaling and model
                    pipeline = ImbPipeline([
                        ('scaler', MinMaxScaler()),  # Standardization
                        ('oversample', SMOTE(random_state=42)),  # Oversample the minority class
                        ('model', model_info['model']) # Model
                    ])
                    
                    # Perform Grid Search with Cross-Validation
                    grid_search = GridSearchCV(
                        estimator=pipeline,
                        param_grid=hyperparams,
                        scoring=balanced_accuracy_scorer, 
                        n_jobs=-1,
                        verbose=1
                    )
                    
                    print(f"Starting Grid Search for {data_type} {time_frame} {model_name} {label_0_name}_vs_{label_1_name}...")
                    grid_search.fit(X_train, y_train)

                    # Best parameters and model
                    best_pipeline = grid_search.best_estimator_
                    best_params = grid_search.best_params_
                    print(f"Best parameters for {model_name} {label_0_name}_vs_{label_1_name}: {best_params}")

                    # # --- SHAP Explainer ---
                    # shap_analysis_pipeline(model = best_pipeline['model'], 
                    #                        X_SHAP = X_SHAP_train, 
                    #                        y_SHAP = np.array(y_train, dtype=bool), 
                    #                        model_savepath = f'{model_save_path}/SHAP_train_val')
                    

                for X_test, y_test, names in BinaryLabelLoader.process_data_stream(df_test, [(label_0, label_1)]):
                    # SHAP Variables
                    X_SHAP_test = X_test.copy(deep=True)

                    X_test = np.array(X_test)
                    y_test = np.array(y_test)

                    # --- Performance Metrics ---
                    # Use cross_val_predict for evaluation
                    y_pred = best_pipeline.predict(X_test)
                    y_pred_prob = best_pipeline.predict_proba(X_test)[:, 1]
                    
                    run_info = {
                        'data_type': data_type,
                        'time_frame': time_frame,
                        'label_0': label_0_name,
                        'label_1': label_1_name,
                        'inputs': input_variable,
                        'model_name': model_name,
                        'best_params': best_params,
                        'set': 'test',
                    }

                    metrics = eval_report(y_test, y_pred, y_pred_prob, model_save_path, run_info)

                    # --- SHAP Explainer ---
                    shap_analysis_pipeline(model = best_pipeline['model'], 
                                           X_SHAP = X_SHAP_test, 
                                           y_SHAP = np.array(y_test, dtype=bool), 
                                           model_savepath = f'{model_save_path}/SHAP_test')

### Test Set Metrics Summary
Create a csv file that compiles the test set performance metrics from each best finetuned model. 

In [ ]:
# --- MODEL SUMMARY (TEST SET) ---
results = []

for time_frame in time_frame_list:
    for data_type in data_type_list:
        analysis_path = save_path_reference.replace('MODEL','classification_1class_meds').replace('INDEPENDENT_VAR',f'v3/{data_type}/{time_frame}') 

        # Load, select, and visualize data
        _, df_data, continuous =  load_data(master_data_path, master_encoded_data_path, analysis_path, data_type, time_frame)
        
        # Prepare data for training
        df_dev, df_test = split_data(df_data, y_col)

        categorical = [col for col in df_data.columns if col not in continuous]
        BinaryLabelLoader = MultiClassToBinaryConverter(y_col, categorical_cols=categorical)
        
        # Iterate over each label pair and predictor variable
        for X_train, y_train, names in BinaryLabelLoader.process_data_stream(df_dev, paired_labels):
            X_train = np.array(X_train)
            y_train = np.array(y_train)
            label1, label2, input_variable = names.values()
            label_name1 = 'rest' if label1 == 'rest' else labels_dict[label1]
            label_name2 = labels_dict[label2] 

            # Directory to save project analysis
            model_save_path = f'{analysis_path}/pred_{label_name1}_vs_{label_name2}/best_model_test_set'
            #print(model_save_path)

            save_path = model_save_path + '/metrics.json'
            metrics_dict = load_json(save_path)
            results.append(metrics_dict)
    
# Convert results to a DataFrame for better visualization
print("Test Set Results:")
results_df = pd.DataFrame(results)
print(len(results_df))
#print(results_df)

# Save tree results summary
comparison_dir = save_path_reference.replace('MODEL','classification_1class_meds').replace('INDEPENDENT_VAR',f'v3')
csv_path = f'{comparison_dir}/tree_models_test_summary_v3.csv'
print('run summary: ', csv_path)
results_df.to_csv(csv_path)

### SHAP Feature Importance Rank Aggregation

In [ ]:
# --- SHAP FEATURE RANK AGGREGATION ---
def rank_data_selection(time_frame_list, data_type_list, paired_labels):
    csv_list = []
    for time_frame in time_frame_list:
        for data_type in data_type_list:
            analysis_path = save_path_reference.replace('MODEL','classification_1class_meds').replace('INDEPENDENT_VAR',f'v3/{data_type}/{time_frame}') 

            # Iterate over each label pair and predictor variable
            for label1, label2 in paired_labels:
                label_name1 = 'rest' if label1 == 'rest' else labels_dict[label1]
                label_name2 = labels_dict[label2] 

                # Directory to save project analysis
                model_save_path = f'{analysis_path}/pred_{label_name1}_vs_{label_name2}/best_model_test_set'

                csv_list.append(f"{model_save_path}/SHAP_test/SHAP_mean_importance.csv")
    return csv_list

def rank_best_data_selection(df, metric='balanced_accuracy_mean'):

    # Find the indices of the rows with the highest balanced_accuracy for each label
    idx = df.groupby('labels')[metric].idxmax()
    # Use these indices to filter the DataFrame
    df = df.loc[idx]

    csv_list = []
    for index, row in df.iterrows():
        time_frame = row['timeframe']
        data_type = row['datatype']
        analysis_path = save_path_reference.replace('MODEL','classification_1class_meds').replace('INDEPENDENT_VAR',f'v3/{data_type}/{time_frame}') 
        
        label_name1 = row['label_0']
        label_name2 = row['label_1']
        # Directory to save project analysis
        model_save_path = f'{analysis_path}/pred_{label_name1}_vs_{label_name2}/best_model_test_set'

        csv_list.append(f"{model_save_path}/SHAP_test/SHAP_mean_importance.csv")
    return csv_list

def rank_prep(file_path, feature_col, metric_col):
    """Process individual CSV file and return normalized Borda scores"""
    # Read CSV
    df = pd.read_csv(file_path)
    
    # Filter out rows with metric_col < 0.01
    df_filtered = df[df[metric_col] >= 0.01]
    
    if df_filtered.empty:
        return pd.Series(dtype=float)
    
    # Sort by metric_col descending
    df_sorted = df_filtered.sort_values(metric_col, ascending=False)
    
    # Assign Borda points (highest metric gets highest points)
    m = len(df_sorted)
    df_sorted['borda'] = range(m, 0, -1)
    
    # Normalize scores to sum to 1
    total_points = df_sorted['borda'].sum()
    df_sorted['normalized'] = df_sorted['borda'] / total_points
    
    return df_sorted.set_index(feature_col)['normalized']

def rank_aggregation(mean_shap_path_list):
        # Process all files and collect scores
    all_scores = []
    all_features = set()

    for file in mean_shap_path_list:
        scores = rank_prep(file, feature_col='Feature', metric_col='Mean_SHAP')
        all_scores.append(scores)
        all_features.update(scores.index.tolist())

    # Create a DataFrame with all cities
    all_features = list(all_features)
    borda_df = pd.DataFrame(index=all_features)

    # Add each person's scores (0 for unranked cities)
    for i, scores in enumerate(all_scores):
        borda_df[f'model_{i+1}'] = scores.reindex(all_features).fillna(0)

    # Calculate total Borda scores
    borda_df['total_score'] = borda_df.sum(axis=1)
    borda_df = borda_df.sort_values(by='total_score' ,ascending=False)
    
    # Sumarize findings
    borda_df.index.name = 'feature'
    borda_df = borda_df.reset_index()
    borda_df['rank'] = np.arange(1,len(borda_df)+1)

    # # Get final ranking
    # final_ranking = borda_df['total_score'].sort_values(ascending=False)
    #print("Final Borda Count Ranking:")
    #print(final_ranking.to_string(float_format='%.3f'))
    
    return borda_df

# List of input CSV files (replace with your actual file paths)
mean_shap_path_list = rank_best_data_selection(df=results_df[results_df['timeframe'] == "2012_to_2024"], metric='balanced_accuracy_mean')
tab_text_ranked_df = rank_aggregation(mean_shap_path_list)
print('best bala acc ranked feature importance')
print(tab_text_ranked_df['feature'][0:10].to_string(float_format='%.3f'))

mean_shap_path_list = rank_data_selection(time_frame_list=['2012_to_2024'], data_type_list=['tabular_text'], paired_labels=paired_labels)
tab_text_ranked_df = rank_aggregation(mean_shap_path_list)
print('tabular_text ranked feature importance')
print(tab_text_ranked_df['feature'][0:10].to_string(float_format='%.3f'))

mean_shap_path_list = rank_data_selection(time_frame_list=['2012_to_2024'], data_type_list=['tabular'], paired_labels=paired_labels)
tab_ranked_df = rank_aggregation(mean_shap_path_list)
print('tabular ranked feature importance')
print(tab_ranked_df['feature'][0:10].to_string(float_format='%.3f'))

mean_shap_path_list = rank_data_selection(time_frame_list=['2012_to_2024'], data_type_list=['text'], paired_labels=paired_labels)
text_ranked_df = rank_aggregation(mean_shap_path_list)
print('text ranked feature importance')
print(text_ranked_df['feature'][0:10].to_string(float_format='%.3f'))